In [ ]:
# Training Notebook
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

base_path = "../data"
image_size = (224, 224)

train_csv = pd.read_csv(os.path.join(base_path, "train_labels.csv"))
train_dir = os.path.join(base_path, "train")

X, y = [], []
for _, row in train_csv.iterrows():
    img = load_img(os.path.join(train_dir, row['image_id']), target_size=image_size)
    img = img_to_array(img) / 255.0
    X.append(img)
    y.append(row['soil_type'])

X = np.array(X)
y = to_categorical(pd.Categorical(y).codes)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
